In [1]:
!pip install ultralytics
!pip install numpy
!pip install ultralytics tensorflow opencv-python-headless matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
!tar -xf images.tar

!wget http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar
!tar -xf annotation.tar

--2025-02-21 04:26:29--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 756.82M  24.3MB/s    in 88s     

2025-02-21 04:27:57 (8.63 MB/s) - ‘images.tar’ saved [793579520/793579520]

--2025-02-21 04:28:00--  http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21852160 (21M) [application/x-tar]
Saving to: ‘annotation.tar’

annotation.tar      100%[===================>]  20.84M  20.8MB/s    in 1.0s    

2025-02-21 04:28:01 (20.8 MB/s) - ‘annotat

In [3]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

In [4]:
annotation_path = '/content/Annotation'
images_path = '/content/Images'
output_path = '/content/yolo_annotations'
os.makedirs(output_path, exist_ok=True)

In [5]:
breed_to_id = {breed: idx for idx, breed in enumerate(os.listdir(annotation_path))}

In [6]:
missing_count = 0
for breed in os.listdir(annotation_path):
    for xml_file in os.listdir(os.path.join(annotation_path, breed)):
        expected_image = xml_file.replace('.xml', '.jpg')
        image_path = os.path.join(images_path, breed, expected_image)

        if not os.path.exists(image_path):
            print(f"Missing: {image_path}")
            missing_count += 1

print(f"Total missing images: {missing_count}")

Missing: /content/Images/n02116738-African_hunting_dog/n02116738_8669
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_607
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_3365
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_4641
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_4732
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_10024
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_1105
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_5936
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_9844
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_5519
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_6283
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_4367
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_2515
Missing: /content/Images/n02116738-African_hunting_dog/n02116738_4461
Missing: /content/Im

In [7]:
#converts XML to Yolo format
def convert_to_yolo(xml_file, img_width, img_height, class_id):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    yolo_annotations = []

    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        try:
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # Convert to YOLO format
            x_center = (xmin + xmax) / 2.0 / img_width
            y_center = (ymin + ymax) / 2.0 / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")
        except Exception as e:
            print(f"Error processing bounding box: {e}. Skipping...")

    return yolo_annotations

In [8]:
for breed in os.listdir(annotation_path):

    breed_path = os.path.join(annotation_path, breed)
    class_id = breed_to_id[breed]

    for xml_file in os.listdir(breed_path):
        xml_file_path = os.path.join(breed_path, xml_file)
        image_file = xml_file+'.jpg'  # Try default extension
        image_path = os.path.join(images_path, breed, image_file)

        # Try multiple possible image extensions
        possible_ext = ['.jpg', '.jpeg', '.png']
        for ext in possible_ext:
            temp_path = os.path.join(images_path, breed, xml_file.replace('.xml', ext))
            if os.path.exists(temp_path):
                image_path = temp_path
                break  # Stop if the correct file is found

        if not os.path.exists(image_path):
            print(f"Warning: Image {image_path} not found. Skipping...")
            continue

        try:
            with Image.open(image_path) as img:
                img_width, img_height = img.size

            # Convert to YOLO format
            yolo_data = convert_to_yolo(xml_file_path, img_width, img_height, class_id)

            if not yolo_data:
                print(f"Warning: No valid annotations for {xml_file_path}. Skipping...")
                continue

            # Save YOLO annotations
            output_file = os.path.join(output_path, f"{image_file.replace('.jpg', '.txt')}")
            with open(output_file, 'w') as f:
                f.write('\n'.join(yolo_data))

        except Exception as e:
            print(f"Error processing file {xml_file_path}: {e}. Skipping...")

print("Finished converting all remaining annotations to YOLO format.")

Finished converting all remaining annotations to YOLO format.


In [9]:
import os
#verifying the correctness of data
# Function to count total files inside subdirectories
def count_files_in_subdirs(main_dir):
    total_files = 0
    for subdir in os.listdir(main_dir):
        subdir_path = os.path.join(main_dir, subdir)
        if os.path.isdir(subdir_path):  # Ensure it's a folder
            total_files += len(os.listdir(subdir_path))
    return total_files

# Count images and annotations
num_images = count_files_in_subdirs('/content/Images')
num_annotations = count_files_in_subdirs('/content/Annotation')
num_yolo_annotations = len(os.listdir('/content/yolo_annotations'))  # YOLO annotations are flat

print("Total images:", num_images)
print("Total annotations:", num_annotations)
print("Total YOLO annotations:", num_yolo_annotations)


Total images: 20580
Total annotations: 20580
Total YOLO annotations: 20580


In [10]:
import os
#update breed label to 0 as we want to detect only dog not the breed
yolo_annotations_path = "/content/yolo_annotations"

# Iterate over all YOLO annotation files
for file_name in os.listdir(yolo_annotations_path):
    file_path = os.path.join(yolo_annotations_path, file_name)

    # Read the existing YOLO annotation
    with open(file_path, "r") as f:
        lines = f.readlines()

    # Update class ID to 0 for all entries
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        parts[0] = "0"  # Set class ID to 0 (representing "dog")
        updated_lines.append(" ".join(parts))

    # Write back the modified annotations
    with open(file_path, "w") as f:
        f.write("\n".join(updated_lines))

print("✅ All labels updated to class 'dog' (class ID = 0).")


✅ All labels updated to class 'dog' (class ID = 0).


In [11]:
import os
import shutil

# Source directory (where images are currently stored in breed-wise folders)
images_source_dir = '/content/Images'  # Contains breed-wise subfolders

# Target directory (where we will move all images)
images_target_dir = '/content/stanford_dogs_yolo/images'  # New flat folder

# Create the target directory if it doesn't exist
os.makedirs(images_target_dir, exist_ok=True)

# Loop through each breed-wise folder and move images
for breed_folder in os.listdir(images_source_dir):
    breed_folder_path = os.path.join(images_source_dir, breed_folder)

    if os.path.isdir(breed_folder_path):  # Check if it's a folder
        for image_file in os.listdir(breed_folder_path):
            src_path = os.path.join(breed_folder_path, image_file)
            dst_path = os.path.join(images_target_dir, image_file)

            # Move image
            shutil.move(src_path, dst_path)

print(f"✅ Moved all images to {images_target_dir}")


✅ Moved all images to /content/stanford_dogs_yolo/images


In [12]:
# Source directory (contains breed-wise annotation subfolders)
annotations_source_dir = '/content/yolo_annotations'

# Target directory (where all YOLO labels will be moved)
labels_target_dir = '/content/stanford_dogs_yolo/labels'
os.makedirs(labels_target_dir, exist_ok=True)

# Move all labels from breed-wise subfolders to a single folder
for breed_folder in os.listdir(annotations_source_dir):
    breed_folder_path = os.path.join(annotations_source_dir, breed_folder)

    if os.path.isdir(breed_folder_path):
        for label_file in os.listdir(breed_folder_path):
            src_path = os.path.join(breed_folder_path, label_file)
            dst_path = os.path.join(labels_target_dir, label_file)

            shutil.move(src_path, dst_path)  # Move annotation

print(f"✅ Moved all labels to {labels_target_dir}")


✅ Moved all labels to /content/stanford_dogs_yolo/labels


In [13]:
import os
import yaml

dataset_dir = '/content/stanford_dogs_yolo'
images_dir = os.path.join(dataset_dir, 'images')  # New flat images folder
labels_dir = os.path.join(dataset_dir, 'labels')  # New flat labels folder
custom_yaml_path = os.path.join(dataset_dir, 'custom.yaml')

def update_yaml():
    os.makedirs(dataset_dir, exist_ok=True)

    custom_yaml = {
        'train': images_dir,  # Correct path to training images
        'val': images_dir,  # (Update this if you want a separate validation set)
        'nc': 1,  # Number of classes (only 'dog')
        'names': ['dog']  # YOLO class names
    }

    # Save the YAML file
    with open(custom_yaml_path, 'w') as yaml_file:
        yaml.dump(custom_yaml, yaml_file, default_flow_style=False)

    print(f"✅ Updated custom.yaml at {custom_yaml_path}")

update_yaml()

✅ Updated custom.yaml at /content/stanford_dogs_yolo/custom.yaml


In [14]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17269 (from 2)
Receiving objects: 100% (17270/17270), 16.12 MiB | 17.84 MiB/s, done.
Resolving deltas: 100% (11858/11858), done.


In [15]:
pip install -r yolov5/requirements.txt

In [16]:
!pip uninstall wandb -y
!rm -rf /root/.config/wandb

Found existing installation: wandb 0.19.6
Uninstalling wandb-0.19.6:
  Successfully uninstalled wandb-0.19.6


In [ ]:
!pip show wandb

In [17]:
mv /content/yolo_annotations/*.txt /content/stanford_dogs_yolo/labels/


In [18]:
!python yolov5/train.py --img 640 --batch 16 --epochs 2 --data /content/stanford_dogs_yolo/custom.yaml --weights yolov5s.pt

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
        0/1      3.55G    0.08807    0.03006          0         38        640:   7% 84/1287 [00:31<05:45,  3.48it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
        0/1      3.55G    0.08768    0.03003          0         44        640:   7% 85/1287 [00:31<06:03,  3.31it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
        0/1      3.55G    0.08737       0.03          0         46        640:   7% 86/1287 [00:31<05:46,  3.47it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(

In [19]:
from google.colab import files
files.download('yolov5/runs/train/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
from google.colab import drive
drive.mount('/content/drive')

# Copy the model to Google Drive
!cp yolov5/runs/train/exp/weights/best.pt /content/drive/MyDrive/

ValueError: mount failed